In [ ]:
import numpy
import pandas as pd
import csv
import time

data_path = './data/'
# read local files

Custom_Header = ["Sentence", "Sentiment"]
data_amazon = pd.read_csv(data_path+'amazon.txt', sep='\t', quoting=csv.QUOTE_NONE,
                          header=None, names=Custom_Header)
# imdb pandas reading only 748 rows of 1000 fixed by adding quoting=csv.QUOTE_NONE
data_imdb = pd.read_csv(data_path+'imdb.txt', sep='\t', quoting=csv.QUOTE_NONE,
                        header=None, names=Custom_Header)
data_yelp = pd.read_csv(data_path+'yelp.txt', sep='\t', quoting=csv.QUOTE_NONE,
                        header=None, names=Custom_Header)

# stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
#               "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
#               "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
#               "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do",
#               "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
#               "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
#               "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
#               "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
#               "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
#               "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
# print(data.head())
# print(data.info())

# loop through data
# for i in range(0, len(data)):
#     if data.iloc[i,1] == 1:
#         #print(data.iloc[i,0], 'Positive')
#         positive = positive + 1
#     else:
#         #print(data.iloc[i,0], 'Negative')
#         negative = negative + 1

# data[column][line] not working after adding column names
# column = {0,1} => get sentence or sentiment
# line = {0,999} => which line
# print(data[0][500])
# Alternative =>
# print(data['Sentence'][500])
# print(data['Sentiment'][500])

# if we use data.iloc[x,y]
# x = line number {0,999}
# y = column number {0,1}
# if y = 0 => get sentence
# if y = 1 => get sentiment
#print(data.iloc[500, 0])


In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words_array = stopwords.words('english') 

In [ ]:

def tokenize_words(data):
    # create an empty list to store the clean words
    clean_words = []
    # loop through every sentence in the data
    for i in range(0, len(data)):
        # remove punctuation from the sentence
        text = data["Sentence"][i]
        text = ''.join([c for c in text if c not in string.punctuation])
        # tokenize the sentence into words
        tokens = word_tokenize(text)
        # remove stop words from the tokens
        tokens = [w for w in tokens if not w in stop_words]
        # append the tokens to the empty list
        clean_words.append(tokens)
    return clean_words

# create a list of tuples containing words in the sentences and their labels
def create_word_features(words):
    my_dict = dict([(word, True) for word in words])
    return my_dict


In [25]:
import string
from nltk.tokenize import word_tokenize
# nltk.download('punkt')
# tokenize the sentences () into words and remove punctuation and stop words
# tokenize the words every data_amazon row using word_tokenize 

alt_start = time.time()
# with list comprehension
alt_tokens_amazon = set(word.lower() for words in data_amazon['Sentence'] for word in word_tokenize(words) if word.lower() not in stop_words_array and word.lower() not in string.punctuation)
alt_end = time.time()

print("Time taken to tokenize amazon data with list comprehension: ", alt_end-alt_start)
for_start = time.time()
# without list comprehension 
amazon_tokens = set()
for words in data_amazon["Sentence"]:
    for word in word_tokenize(words):
        if word.lower() not in stop_words:
            if word.lower() not in string.punctuation:
                amazon_tokens.add(word.lower())
for_end = time.time()
print("Time taken to tokenize amazon data without list comprehension: ", for_end-for_start)

Time taken to tokenize amazon data with list comprehension:  0.14963722229003906
Time taken to tokenize amazon data without list comprehension:  0.1040182113647461


Time taken to tokenize amazon data with list comprehension:  0.19963722229003906
Time taken to tokenize amazon data without list comprehension:  0.1040182113647461

In [ ]:
alt_start = time.time()
amazon_train = [({word: (word in word_tokenize(data_amazon['Sentence'][i].lower()) and word.lower() not in stop_words_array)
                  for word in amazon_tokens}, data_amazon['Sentiment'][i]) for i in range(0, len(data_amazon))]
alt_end = time.time()
print("Time taken to create training data with list comprehension: ", alt_end-alt_start)

# print(len(amazon_train))
# print(amazon_train[0])
for_start = time.time()
amazon_train_alt = []
for i in range(0, len(data_amazon)):
    # This dictionary with the word as key and True if the word is in the word_tokenize and False if it is not
    amazon_dict = {}
    # we compare each word in the amazon_tokens to each word in the data_amazon['Sentence'][i] and if the word is in the data_amazon['Sentence'][i]
    # we add it to the dictionary with the value True and if it is not in the data_amazon['Sentence'][i] we add it to the dictionary with the value False
    for word in amazon_tokens:
        if word in word_tokenize(data_amazon['Sentence'][i].lower()) and word.lower() not in stop_words_array and word not in string.punctuation:
            amazon_dict[word] = True
        else:
            amazon_dict[word] = False
    # create a tuple with the dictionary and the sentiment
    amazon_tuple = (amazon_dict, data_amazon['Sentiment'][i])
    # add the tuple to the amazon_train list
    amazon_train_alt.append(amazon_tuple)
for_end = time.time()
print("Time taken to create training data without list comprehension: ",
      for_end-for_start)

# print(len(amazon_train_alt))
# print(amazon_train_alt[0])


Time taken to create training data with list comprehension:  176.76983428001404
Time taken to create training data without list comprehension:  178.1755678653717


In [ ]:
import random
random.shuffle(amazon_train)
amazon_train_x = amazon_train[:int(len(amazon_train)*0.8)]

In [ ]:
import nltk
#create model using  NaiveBayesClassifier
model = nltk.NaiveBayesClassifier.train(amazon_train_x)
model.show_most_informative_features()


In [ ]:
random.shuffle(amazon_train)
amazon_test_x = amazon_train[int(len(amazon_train)*0.2):]
acc=nltk.classify.accuracy(model, amazon_test_x)
print("Accuracy:", acc)

In [ ]:
tests=['I really like it', 
       'I do not think this is good one', 
       'this is good one',
       'I hate the show!']
for test in tests:
    print(test, model.classify({word: (word in word_tokenize(test.lower()))
                 for word in amazon_tokens}))
                 